In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [5]:
from loguru import logger
from langchain.callbacks import FileCallbackHandler
from datetime import date

today = date.today()

logfile = f"logs/full_pipeline_{today.year}_{today.month}_{today.day}.log"
logger.add(logfile, format="{message}", colorize=False, enqueue=True)
handler = FileCallbackHandler(logfile)

In [ ]:
from lib import create_cube_selection_chain, fetch_cubes_descriptions, parse_all_cubes, fetch_cube_sample, fetch_dimensions_triplets

cube_selection_settings = {
    "temperature": 0.5,
    "top_p": 0.5
}
cubes = fetch_cubes_descriptions()
#dimensions = fetch_dimensions()
cube_selection_chain = create_cube_selection_chain(api_key=OPENAI_API_KEY, handler=handler, **cube_selection_settings)

questions = [
"sum of emission of CO2 for industry between year 2009 and 2011",
"get average of emission of Methane for transport between years 2007 and 2005",
"What percentage of emission was from N2O and CH4 compared to total emission?",
"what bathing stations are in switzerland?",
"what swiss bathing stations had poor water quality in 2024?",
"what is maximum contamination in lead in soil in 2022?",
"what's for lunch"]

question=questions[-2]


cube_selection_response = await cube_selection_chain.ainvoke({
    "cubes": cubes,
    "question": question,
})
cube_selection_response = cube_selection_response['text']

logger.info("========== CUBES RESPONSE ================")
logger.info(f"{cube_selection_response}")

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
2025-01-16 09:58:21.775 | INFO     | __main__:<module>:29 - ========== CUBES RESPONSE ================
2025-01-16 09:58:21.777 | INFO     | __main__:<module>:30 - The cube that best answers the question regarding the percentage of emissions from N2O (nitrous oxide) and CH4 (methane) compared to total emissions is:

**Cube ID:** <https://environment.ld.admin.ch/foen/ubd000502/5>

**Justification:** This cube titled "Greenhouse gas emissions by sector and by gas" provides information on the evolution of Switzerland’s greenhouse gas emissions broken down by gas. It is likely to include data on N2O and CH4 emissions as part of the total greenhouse gas emissions, which is essential for calculating the percentage of these specific gases in relation to total emissions.

### Available Topics in the Cubes:
- Forest fire danger
- 

In [7]:
question

'what is maximum contamination in lead in soil in 2022?'

In [8]:
selected_cubes = parse_all_cubes(cube_selection_response)
selected_cube = selected_cubes[0]

logger.info("========== SELECTED FIRST CUBE ================")
logger.info(selected_cube)

2025-01-16 09:53:23.391 | INFO     | __main__:<module>:4 - ========== SELECTED FIRST CUBE ================
2025-01-16 09:53:23.393 | INFO     | __main__:<module>:5 - <https://environment.ld.admin.ch/foen/ubd006601/3>


In [9]:
cube_and_sample = fetch_cube_sample(selected_cube)

# print(cube_and_sample)

In [10]:
dimensions_triplets = fetch_dimensions_triplets(selected_cube)

# print(dimensions_triplets)

In [11]:
from lib import create_query_generation_chain

query_generation_settings = {
    "temperature": 0.2,
    "top_p": 0.1
}

generation_chain = create_query_generation_chain(api_key=OPENAI_API_KEY, handler=handler, **query_generation_settings)

query_generation_response = await generation_chain.ainvoke({
    "cube_and_sample": cube_and_sample,
    "dimensions_triplets": dimensions_triplets,
    "cube": selected_cube,
    "question": question,
})
query_generation_response = query_generation_response['text']

logger.info("========== QUERY GENERATION RESPONSE ================")
logger.info(f"{query_generation_response}")

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
2025-01-16 09:53:33.429 | INFO     | __main__:<module>:18 - ========== QUERY GENERATION RESPONSE ================
2025-01-16 09:53:33.431 | INFO     | __main__:<module>:19 - PREFIX cube: <https://cube.link/>
PREFIX schema: <http://schema.org/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT MAX(?wert) AS ?maxLeadContamination
WHERE {
    <https://environment.ld.admin.ch/foen/ubd006601/3> a cube:Cube;
        cube:observationSet ?observationSet.

    ?observationSet a cube:ObservationSet;
        cube:observation ?observation.

    ?observation a cube:Observation;
        <https://environment.ld.admin.ch/foen/ubd006601/we

In [12]:
from lib import run_query

result = run_query(query_generation_response)

logger.info("=========== QUERY RESULT ============")
logger.info(result)

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'{"message":"com.complexible.stardog.plan.eval.ExecutionException: Invalid SPARQL query: Encountered \\" \\"max\\" \\"MAX \\"\\" at line 9, column 8.\\nWas expecting one of:\\n    \\"(\\" ...\\n    \\"*\\" ...\\n    \\"reduced\\" ...\\n    \\"distinct\\" ...\\n    <VAR1> ...\\n    <VAR2> ...\\n    ","code":"QE0PE2","timestamp":"2025-01-16T08:53:33.571606Z","stacktrace":"com.stardog.stark.query.QueryExecutionFailure: com.complexible.stardog.plan.eval.ExecutionException: Invalid SPARQL query: Encountered \\" \\"max\\" \\"MAX \\"\\" at line 9, column 8.\\nWas expecting one of:\\n    \\"(\\" ...\\n    \\"*\\" ...\\n    \\"reduced\\" ...\\n    \\"distinct\\" ...\\n    <VAR1> ...\\n    <VAR2> ...\\n    \\n\\tat com.stardog.stark.query.QueryExecutionFailure.forLoggedError(QueryExecutionFailure.java:46)\\n\\tat com.complexible.stardog.query.DefaultQueryFactory$Se